In [1]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import pandas as pd
from netCDF4 import Dataset
import numpy as np

In [2]:
def date_column_to_matrix(subset_time, rows, columns):
    date=pd.to_datetime(subset_time)
    start = date.min().timestamp()
    end = date.max().timestamp()
    a = np.linspace(start, end, rows)
    b = a.reshape(-1,1)
    c = np.ones((1,columns))
    d = b * c
    return d

In [3]:
def main_function(fh):
    #Get Values from file for lat, lon, N0" and time
    no2_2 = fh.groups['PRODUCT'].variables['nitrogendioxide_tropospheric_column'][:]
    no2_2[fh.groups['PRODUCT'].variables['qa_value'][:,:]<0.75]=np.nan
    lons = fh.groups['PRODUCT'].variables['longitude'][:]
    lats = fh.groups['PRODUCT'].variables['latitude'][:]
    qa=fh.groups['PRODUCT'].variables['qa_value'][:]
    time = fh.groups['PRODUCT'].variables['time_utc'][:]
    subset_lons = lons[0,:,:]
    rows, columns = subset_lons.shape
    subset_lats = lats[0,:,:]
    subset_no2 = no2_2[0,:,:]
    subset_time = time[0, :]
    subset_time_new = date_column_to_matrix(subset_time, rows, columns)
    flatten_lon = subset_lons.flatten()
    flatten_lat = subset_lats.flatten()
    flatten_no2 = subset_no2.flatten()
    flatten_time = subset_time_new.flatten()
    df = pd.DataFrame({'longitudes': flatten_lon,
                      'latitudes': flatten_lat,
                      'NO2': flatten_no2,
                      'time': flatten_time})
    df = df.dropna()
    df1 = df[(df['longitudes'] >=-12.0) & (df['longitudes'] <=5.0) & (df['latitudes'] >=35) & (df['latitudes'] <=44.0)]
    df1['date'] = pd.to_datetime(df1['time'], unit='s')
    df1['file'] = str(os.fsdecode(file))
    return df1

In [4]:
import os

path = '/Users/aitorizquierdo/Desktop/air-quality-madrid/sentinel/'

folder = os.fsencode(path)

final_df = []

for file in os.listdir(folder):
    filename = os.fsdecode(file)
    if filename.endswith( ('.nc') ): # whatever file types you're using...
        #print(os.fsdecode(file))
        my_example_nc_file = path+os.fsdecode(file)
        fh = Dataset(my_example_nc_file, mode='r')
        temp_df = main_function(fh)
        final_df.append(temp_df)
        #print(final_df)
final_df = pd.concat(final_df,axis=0)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [7]:
export_csv = final_df.to_csv (path+'\export_dataframe.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path